In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import seaborn as sns 
import matplotlib.pyplot as plt 

In [2]:
RANDOM_STATE = 42

# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
train.head()

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [3]:
train = train.drop_duplicates()

In [4]:
train['target'].value_counts()

target
Normal      38150
AbNormal     2350
Name: count, dtype: int64

In [5]:
for i in train.columns:
    if train[i].isnull().sum() == len(train):
        train = train.drop(i,axis=1)
        test = test.drop(i,axis=1)

In [6]:
test = test.drop('Set ID',axis=1)

In [7]:
for i in train.columns:
    if 'OK' in list(train[i].value_counts().keys()):
        train = train.drop(i,axis=1)
        test = test.drop(i,axis=1)

In [8]:
lst = []
for i in test.columns:
    x = 0
    lens = len(test[i].value_counts().keys())
    for value in test[i].value_counts().keys():
        if value in train[i].value_counts().keys():
            x += 1
    if x == lens:
        lst += [i]

In [9]:
train = train[lst]
test = test[lst[:-1]]

In [10]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40500 entries, 0 to 40505
Columns: 151 entries, Wip Line_Dam to target
dtypes: float64(64), int64(70), object(17)
memory usage: 47.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 150 entries, Wip Line_Dam to WorkMode Collect Result_Fill2
dtypes: float64(64), int64(70), object(16)
memory usage: 19.9+ MB


In [11]:
nowsss = 13

print(nowsss*2725,(nowsss+1)*2725)
df_normal = train[train["target"] == "Normal"][nowsss*2725:(nowsss+1)*2725]
df_abnormal = train[train["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

35425 38150
  Total: Normal: 2725, AbNormal: 2350


target
Normal      2725
AbNormal    2350
Name: count, dtype: int64

In [12]:
df = df_concat
df_test = test

In [13]:
for i in df.columns:
    if len(df[i].value_counts().keys()) == 1:
        df = df.drop(i,axis=1)
        df_test = df_test.drop(i,axis=1)

In [14]:
df['target'] = np.where(df['target'] == 'Normal',0,1)

In [15]:
df = df.drop([i for i in df.select_dtypes('object').columns if 'Equipment_' in i],axis=1)
df_test = df_test.drop([i for i in df.select_dtypes('object').columns if 'Equipment_' in i],axis=1)

In [16]:
l = []

for col in list(df.select_dtypes('object').columns):
    per0 = []
    per1 = []
    per = []

    for i in list(df[col].value_counts().keys()):
        a = len(df[df[col] == i])
        b = len(df[(df[col] == i) & (df['target'] == 1)])
        per += [round(b/a,1)]

        if (round(b/a,1)) >= 0.5:
            per0 += [i]
        else:
            per1 += [i]
    
    df[col] = np.where(df[col].isin(per0),1,0)
    df_test[col] = np.where(df_test[col].isin(per0),1,0)

In [17]:
df.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5075 entries, 0 to 5074
Columns: 120 entries, Model.Suffix_Dam to target
dtypes: float64(60), int32(5), int64(55)
memory usage: 4.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 122 entries, Equipment_Dam to WorkMode Collect Result_Fill2
dtypes: float64(60), int32(4), int64(55), object(3)
memory usage: 15.9+ MB


In [18]:
df_test = df_test[df.columns[:-1]]

In [19]:
df.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5075 entries, 0 to 5074
Columns: 120 entries, Model.Suffix_Dam to target
dtypes: float64(60), int32(5), int64(55)
memory usage: 4.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 119 entries, Model.Suffix_Dam to WorkMode Collect Result_Fill2
dtypes: float64(60), int32(4), int64(55)
memory usage: 15.5 MB


In [20]:
l = []

for col in df.columns:
    if len(df[col].value_counts().keys()) >= 3:
        per0 = []
        per1 = []
        per = []

        for i in list(df[col].value_counts().keys()):
            a = len(df[df[col] == i])
            b = len(df[(df[col] == i) & (df['target'] == 1)])
            per += [round(b/a,1)]

            if (round(b/a,1)) >= 0.5:
                per0 += [i]
            else:
                per1 += [i]

        if len([i for i in np.unique(per) if i >= 0.5]) >= 1:
            df[col] = np.where(df[col].isin(per0),1,0)
            df_test[col] = np.where(df_test[col].isin(per0),1,0)
            l += [col]

In [21]:
xtrain = df.drop(['target'],axis=1)
ytrain = df['target']
xtest = df_test

In [22]:
xtrain,xval,ytrain,yval = train_test_split(xtrain,ytrain,test_size=0.2,random_state=RANDOM_STATE)

In [23]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=RANDOM_STATE)
rfc.fit(xtrain,ytrain)

importance = rfc.feature_importances_

# 정렬과 시각화를 쉽게 하기 위해 series 전환
ft_series = pd.Series(importance, index = xtrain.columns)
ft_top20 = pd.DataFrame(ft_series.sort_values(ascending=False)).reset_index()

In [24]:
from sklearn.inspection import permutation_importance

model = RandomForestClassifier(random_state=RANDOM_STATE).fit(xtrain, ytrain)

pfi = permutation_importance(model, xval, yval,n_repeats=10,random_state=RANDOM_STATE)

pfi_df = pd.DataFrame({
    'Feature': xval.columns,
    'Importance Mean': pfi.importances_mean,
    'Importance Std': pfi.importances_std
})

pfi_df = pfi_df.sort_values(by='Importance Mean', ascending=False)

In [25]:
num = 0

for i in range(100):
    l2 = ft_top20['index'][:i]
    l3 = pfi_df['Feature'][:i]
    res = set(l2).intersection(l3)
    if (len(list(res))) >= 60:
        print(i,len(list(res)))
        num = i
        break

86 60


In [26]:
l2 = ft_top20['index'][:num]
l3 = pfi_df['Feature'][:num]

res = list(set(l2).intersection(l3))[:60]

In [27]:
2725 * (nowsss + 1)

38150

In [28]:
best_cols = pd.read_csv('이진분류기준칼럼.csv')
best_cols[f'{2725 * (nowsss + 1)}'] = res

best_cols.to_csv('이진분류기준칼럼.csv',index=False)